In [151]:
using Revise, Plots, FFTW, LinearAlgebra, LaTeXStrings, Random, Distributions, Images, Printf

In [152]:
includet("../src/SparseSpikes.jl")
using .SparseSpikes


Define ground truth amplitudes and positions

In [153]:
domain = [[0, 1], [0, 1]]

λ = 0.032 # Regularisation parameter

0.032

In [154]:
# Define the plot
num_points = 64

plt_grid_x1 = [domain[1][1] + i * (domain[1][2] - domain[1][1]) / (num_points - 1) for j in 0:num_points-1, i in 0:num_points-1]
plt_grid_x2 = [domain[2][1] + j * (domain[2][2] - domain[2][1]) / (num_points - 1) for j in 0:num_points-1, i in 0:num_points-1]

grid = range(0, stop=1, length=num_points)
plot_size = (400, 250) .* 2
# plt = heatmap(xlims=domain[1], ylims=domain[2], xlabel="x Position", ylabel="y Position", color=:viridis, colorbar_title="Amplitude", size=plot_size, grid=false)

# plt_gt = deepcopy(plt)
# plot_spikes!(plt_gt, μ0, label=L"μ_0", colorscheme=:viridis)

(800, 500)

## Check $\sigma$ parameter

In [155]:
# const noise_mean = 0.0021172377176794793
const σ2 = let
    λ = 723.0 # Wavelength
    NA = 1.4 # Numerical aperture
    FWHM = λ / (2 * NA) # Full width at half maximum i.e. diffraction limit
    σ = FWHM / (2 * log(2.0))
    (σ / (64 * 100.0))^2
end
σ = sqrt(σ2)

ops = gaussian_operators_2D(σ, plt_grid_x1, plt_grid_x2)

# plt_test_σ = deepcopy(plt)
# heatmap!(plt_test_σ, grid, grid, ops.Φ(0.5, 0.5, 1.0), color=:viridis)

Operators2D(Main.SparseSpikes.var"#ϕ#60"{Float64, Matrix{Float64}, Matrix{Float64}, Main.SparseSpikes.var"#gauss2DN#59"{Main.SparseSpikes.var"#gauss2D#58"}}(0.029103474178647337, [0.0 0.015873015873015872 … 0.9841269841269841 1.0; 0.0 0.015873015873015872 … 0.9841269841269841 1.0; … ; 0.0 0.015873015873015872 … 0.9841269841269841 1.0; 0.0 0.015873015873015872 … 0.9841269841269841 1.0], [0.0 0.0 … 0.0 0.0; 0.015873015873015872 0.015873015873015872 … 0.015873015873015872 0.015873015873015872; … ; 0.9841269841269841 0.9841269841269841 … 0.9841269841269841 0.9841269841269841; 1.0 1.0 … 1.0 1.0], Main.SparseSpikes.var"#gauss2DN#59"{Main.SparseSpikes.var"#gauss2D#58"}(Main.SparseSpikes.var"#gauss2D#58"())), Main.SparseSpikes.var"#13#23"{Main.SparseSpikes.var"#ϕ#60"{Float64, Matrix{Float64}, Matrix{Float64}, Main.SparseSpikes.var"#gauss2DN#59"{Main.SparseSpikes.var"#gauss2D#58"}}}(Main.SparseSpikes.var"#ϕ#60"{Float64, Matrix{Float64}, Matrix{Float64}, Main.SparseSpikes.var"#gauss2DN#59"{Main.

In [156]:
# plt_obs = deepcopy(plt)

function readImages(imageDir, nImages, n_pixels_x)
    imA = Array{Float64}(undef, n_pixels_x, n_pixels_x, nImages)
    for q = 1:nImages
        imageId = @sprintf("%05d", q)
        img = load("$imageDir/$imageId.tif")
        imA[:, :, q] = channelview(img)
    end
    imA
end

nImages = 10
images = readImages("../sequence", nImages, 64)

64×64×10 Array{Float64, 3}:
[:, :, 1] =
 0.00207523  0.00224308  0.00198367  …  0.00234989  0.00195315  0.00181582
 0.00204471  0.00213626  0.00207523     0.00204471  0.00224308  0.00216678
 0.002121    0.00195315  0.00238041     0.00238041  0.00177005  0.00224308
 0.00184634  0.00257877  0.00199893     0.00225834  0.00227359  0.00227359
 0.0019379   0.00224308  0.00205997     0.0019379   0.00180056  0.00205997
 0.00221256  0.00184634  0.0021973   …  0.00204471  0.00231937  0.00189212
 0.00196841  0.00184634  0.00199893     0.00215152  0.00256352  0.00213626
 0.00213626  0.00222782  0.00241093     0.00215152  0.00190738  0.00177005
 0.00215152  0.00234989  0.00236515     0.00178531  0.00190738  0.00205997
 0.00198367  0.00257877  0.00184634     0.00248722  0.00248722  0.00224308
 ⋮                                   ⋱                          
 0.00170901  0.00225834  0.00209049  …  0.00178531  0.00202945  0.00163272
 0.00180056  0.00260929  0.0019379      0.00199893  0.00175479  0.0018

### SFW

In [157]:
function runsfw(images, ops, λ, domain)
    results = []
    for q in 1:size(images, 3)
        y = vec(images[:, :, q])
        prob = BLASSO(y, ops, λ, domain)
        solve!(prob, :SFW, options=Dict(:maxits => 100))
        push!(results, prob.μ)
    end
    results
end

results = runsfw(images, ops, λ, domain)

10-element Vector{Any}:
 DiscreteMeasure([[0.8492063492063487, 0.19841269841269774, 0.2499999999999992, 0.2499999999999996, 0.29999999999999993], [0.3571428571428571, 0.4999999999999998, 0.2999999999999992, 0.25, 0.5499999999999999]], [0.002216915912520515, 0.0012279995987538742, 0.0004647635444217828, 0.0002703011696800991, 0.000189011757690114], 2, 5)
 DiscreteMeasure([[0.15079365079365029, 0.6904761904761901, 0.29365079365079366, 0.29365079365079344, 0.5476190476190468, 0.15, 0.1], [0.4999999999999996, 0.547619047619081, 0.8015873015873013, 0.7063492063492063, 0.6587301587301585, 0.55, 0.5]], [0.0006438173292435381, 0.0013728043734308022, 0.0010389886006893884, 0.00099238372320154, 0.0012075357408850523, 0.00019450778511450684, 0.00013615754960540332], 2, 7)
 DiscreteMeasure([[0.2936507936507921, 0.5476190476190494, 0.3571428571428681], [0.49999999999999994, 0.5476190476190471, 0.8015873015873016]], [0.0012731303798332108, 0.0006596309728874807, 0.0008743566550918629], 2, 3)
 Discre

In [158]:
function writeCSV(filename, results)
    header = "Ground-truth,frame,xnano,ynano,znano,intensity\n"
    csvfile = open(filename, "w")
    write(csvfile, header)
    for frame in 1:length(results)
        μ = results[frame]
        if μ.dims == 2
            nSources = length(μ.a)
            for i in 1:nSources
                xPos = @sprintf("%.2f", μ.x[1][i] * 6400.0)
                yPos = @sprintf("%.2f", μ.x[2][i] * 6400.0)
                write(csvfile, "$i, $frame, $xPos, $yPos, 0.00, 0.00\n")
            end
        else
            error("Unsupported dimensions")
        end
    end
    close(csvfile)
end

writeCSV (generic function with 1 method)

In [159]:
writeCSV("results.csv", results)